### Auto Approval Cron Job

1. Settings
    1. account ID : accID
    1. vehicle ID : vehID
    1. region
    1. Time range: timeRange
1. Get relevant alerts
    1. Fetch by {accID, timeRange} -- via API
    1. Filter alerts that have remarks/approval 
    1. Filter alert by vehID
1. For each alert get video from aws -- parallel thread
1. Eval for approval <bf>evalApproval(localVideoFile, alertType)</bf>
1. Log results:
    1. [alert_id, image, type, result, stat, run_ts, run_times]
    1. 12345678, <image>, seatbelt, approve, 867/1000, 12345678, (1, 1.3)
    1. 12345678, <image>, mobile, ignore, 67/300, 12345679, (1, 1.3)
1. Update alerts via API


In [78]:
import json, glob, os, sys
from datetime import datetime
import time
import utilsAWS as uAws;
import intanglesAPIs as iApis
import imageio
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
reload(uAws)
uAws.gVerb = False

from PIL import Image
import io
import base64

def getBase64Image(img):
    # Convert NumPy array to PIL Image
    pil_image = Image.fromarray(img)

    # Save the PIL Image to a BytesIO buffer in PNG format
    buffer = io.BytesIO()
    pil_image.save(buffer, format="PNG")
    image_bytes = buffer.getvalue()

    # Base64 encode the image bytes
    base64_encoded_image = base64.b64encode(image_bytes).decode('utf-8')
    return base64_encoded_image

### 1. Settings

In [232]:
settings = {
    'accID': '621348950418460672', # Shrinath
#     'accID': '1313274493921132544', # TMS US
    'tStart':  datetime(2025,7,9,1),
    'tEnd': datetime(2025,7,9,13),
    'region': 'India',
#     'region': 'US',
    'vehID': None,
    'procDir': './AutoApprovalProc/',
}
settings['fetchAlertTypes'] = 'mobile_phone,seatbelt_violation'

vidType = {
    'mobile_phone': 'driver',
    'seatbelt_violation': 'driver',
}

 
if settings['region'].lower()=='us':
    urlUS = 'https://apis.intangles-aws-us-east-1.intangles.us'
    tokenUS = 'FCNlIrz1dHlHyd_QYxF9V8bmLLkxNm_IzA3vq73MXdhpqOcXPf1YGBNIlgnYvrRI'
    settings['urlFetchAlerts'] = urlUS
    settings['urlUpdateAlerts'] = 'https://apis.intangles-aws-us-east-1.intangles.us/alertlog/{alertID}/updateV2?&acc_id={accID}&token={token}'
    settings['token'] = tokenUS
    uAws.initiate_s3_session(isUS=True)
else:
    settings['urlFetchAlerts'] = 'https://apis.intangles.com'
    settings['urlUpdateAlerts'] = 'https://apis.intangles.com/alertlog/{alertID}/updateV2?&acc_id={accID}&token={token}'
    settings['token'] = 'JBRHjp1tPFdyZwvRGblGwi-hIv5OmYu-cr--qzRE9rSCY6F1M5vxQt5Y7Wn9g7ur' 
    uAws.initiate_s3_session(isUS=False)



In [218]:
    
info = {
    'start': datetime.now(),
    'settings': settings,
}

### 2. Fetch Alerts & Apply Filters

In [219]:
t1 = time.time()
dAlerts, info = iApis.getVTAlerts(st = settings['tStart'],
                            en = settings['tEnd'],
                            baseUrl = settings['urlFetchAlerts'],
                            utoken = settings['token'],
                            alertType= settings['fetchAlertTypes'],
                            accID = settings['accID'],
                           )
t2 = time.time()
info['t_fetchAlerts'] = t2-t1
info['n_alerts'] = len(dAlerts)
print("#Alerts=%d, took %d sec"%(len(dAlerts), t2-t1))



#Alerts=1110, took 0 sec


In [220]:
from collections import Counter
Counter([x['type'] for x in dAlerts])

Counter({'mobile_phone': 1098, 'seatbelt_violation': 12})

In [224]:
settings['tEnd']

datetime.datetime(2025, 7, 9, 13, 0)

In [228]:
alertsToProcess = []
info['alertsToProcess'] = []
info['alertsToProcessMeta'] = []
info['alertsDropped'] = []
for ent in dAlerts:
    # filter to keep entries in vehID only (if given)
    vid = ent['vehicle_id']
    if settings['vehID'] is not None:
        if vid not in settings['vehID']:
            continue
    if ent.get('alert_values',{}).get('approval_status',None) is not None:
        continue
        
    # check if video is available
    typ = ent['type']
    vidUrl = None
    for url in ent.get('alert_values',{}).get('artefacts',[]):
        if url.find(vidType[typ]+'.mp4')>=0:
            vidUrl = url
    if vidUrl is None:
        info['alertsDropped'].append((ent['id'], typ, 'no video file'))
    else:
        info['alertsToProcess'].append((ent['id'], typ, vidUrl))
        info['alertsToProcessMeta'].append(ent)
print('#dropped=',len(info['alertsDropped']))
print('#for process=',len(info['alertsToProcess']))

#dropped= 5
#for process= 97


In [230]:
Counter([x[1] for x in info['alertsToProcess']])

Counter({'mobile_phone': 95, 'seatbelt_violation': 2})

### Get Video from S3

In [237]:
videoDir = settings['procDir']+'/vids_%s/'%(datetime.now().strftime('%Y%m%d_%H%M'))
print(videoDir)
os.makedirs(localDir, exist_ok=True)

files = []
for (aId, typ, vidUrl) in info['alertsToProcess']:
    files.append(vidUrl)
#     if len(files)>40:
#         break
    
t1 = time.time()
uAws.downloadFiles(files, outDirPath=videoDir)
t2 = time.time()
info['t_AWS'] = t2-t1

### Process each video via Dino

In [273]:
# initialize
import utilsAutoApprovalDino as uDino
reload(uDino)
uDino.init('AutoApprovalProc/assets/refData.json')

loading DINO model ...


Using cache found in C:\Users\Vikram.Melapudi/.cache\torch\hub\facebookresearch_dinov2_main


loading reference dataset ...
mobile_phone (137, 768)
seatbelt_violation (235, 768)
DONE.


In [234]:
counts = {'Approve':0, 'Ignore':0, 'Verify': 0}
processedAlerts = {}

In [240]:
tempDir = os.path.join(settings['procDir'],'temp/')
os.makedirs(tempDir, exist_ok=True)
    
nVid = 0
for (aId, typ, vidUrl) in info['alertsToProcess']:
    if aId in processedAlerts:
        continue
    nVid += 1
    print('process %d/%d...'%(len(processedAlerts), len(info['alertsToProcess'])))
    t1 = time.time()
    localVidFile = uAws.s3ToLocalPath(vidUrl)
    vid = imageio.get_reader( os.path.join(videoDir,localVidFile) )
    N = 10 # vid.count_frames()
    img = vid.get_data(N//2)
    
    # handle full cabin image -> crop to corrrect half side
    if img.shape[1]>800:
        if settings['region'].lower()=='us':
            img = img[:,img.shape[1]//2:]
        else:
            img = img[:,:img.shape[1]//2]

    qobj = uDino.procImage(img, saveNpz=False)#, forceLoad=True)
    dists, indxs = qobj['knn'].kneighbors(uDino.feats[typ], 3)

    xyDists = [] # tuple of (x,y,D) for each reference patch
    for n in range(len(dists)):
        for m in range(3):
            x, y = uDino.index2xy(indxs[n,m], qobj)
            xyDists.append((x,y,dists[n,m]))
    xyDists = np.array(xyDists) 
    rules = uDino.refDataset.get(typ,{}).get('rules',{})
    # indx = np.where(xyDists[:,2]<rules['threshold'])[0]

    t2 = time.time()

    Nthresh = rules['passThreshold']
    Ntot = len(xyDists)
    Npass = np.sum(xyDists[:,2]<Nthresh)
    if Npass > Ntot//2+int(Ntot*rules['unsureFraction']):
        Nres = rules["passAction"]
    elif Npass < Ntot//2-int(Ntot*rules['unsureFraction']):
        Nres = rules["failAction"]
    else:
        Nres = 'Verify'

    print(qImageFile, typ, Ntot, Npass, Nres, '(%d s)'%(t2-t1))
    counts[Nres] += 1

    rimg = uDino.cv2.resize(img, (150,100))
    
    processedAlerts[str(aId)] = {
        'id': str(aId),
        'img': getBase64Image(rimg),
        'type': str(typ),
        'result': Nres,
        'Npass': Npass,
        'Ntot': Ntot,
        'runS': (t2-t1),
        'date': datetime.now()
    }

    if False:
        plt.figure(figsize=(7,3))
        plt.subplot(121)
        plt.imshow(qobj['image'])
    #     plt.plot(xyDists[indx,0], xyDists[indx,1], 'rx')
        plt.subplot(122)
        plt.plot(sorted(xyDists[:,2]))
        plt.hlines(45,0,len(xyDists))#,'r-')
        plt.xlabel('feat-index')
        plt.ylabel('NN-dist')
        plt.tight_layout()
        plt.show()
print('')
print('DONE.')

process 1/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 322 Approve (2 s)
process 2/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 357 Approve (2 s)
process 3/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 333 Approve (2 s)
process 4/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 284 Approve (2 s)
process 5/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 302 Approve (2 s)
process 6/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 314 Approve (2 s)
process 7/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 321 Approve (2 s)
process 8/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 345 Approve (2 s)
process 9/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 279 Approve (2 s)
process 10/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 314 Approve (2 s)
process 11/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 275 Approve (2 s)
process 12/97...
./AutoApprovalProc/temp/tmp.png mobile_phone 411 305 Approve (2 s)
p

### Write a Summary File

In [242]:
htmlFile = settings['procDir']+'/summary'+settings['region']+'.html'
print('HTML file=', htmlFile)
with open(htmlFile, 'w') as f:
    f.write('''<html><head>
    
    <style>
    *{
      font-family: Arial;
      font-size: 12px;
      }
    th, td {
      padding-left: 10px;
      padding-right: 10px;
    }
    .approve {
        background-color:#DDFFDD;
        padding:5px;
        border-radius:8px;
    }
    .ignore {
        background-color:#FFDDDD;
        padding:5px;
        border-radius:8px;
    }
    .verify {
        background-color:#FFF955;
        padding:5px;
        border-radius:8px;
    }
    </style>
    <script>
    var settingsJSON = '%s'
    function clicked(el) {
      var els = document.getElementsByClassName(el.id);
      console.info(el.id+','+els.length);
      var dval = 'none';
      if(el.checked) dval = '';
      for(var n=0; n<els.length; n++) {
        if(els[n].tagName=='LABEL') continue;
        els[n].style.display = dval;
      }
    }
    </script>
    </head>
    <body style="text-align:center;">
    <h2 style="font-size:24px;background-color:#BBBBFF; padding:5px;">Auto Approval Review</h2>
    <div style="margin:10px"> accID=%s, vIDs=%s, time=[%s, %s]
    </div>
    <div style="margin:10px">
    <input type="checkbox" id="approve" onclick="clicked(this)"><label for="approve" id="lapprove" class="approve">approve</label>
    <input type="checkbox" id="ignore" onclick="clicked(this)"><label for="ignore" id="lignore" class="ignore">ignore</label>
    <input type="checkbox" id="verify" onclick="clicked(this)"><label for="verify" id="lverify" class="verify">verify</label>
    </div>
    <table style="margin:5px; margin-left:auto; margin-right:auto;">
    '''%(json.dumps(settings,default=str), 
         settings['accID'], str(settings['vehID']), 
         str(settings['tStart']), str(settings['tEnd'])) )
    
    header = ['Alert ID','Image','Type','Result','Score','Timing','Date']
    f.write('<tr>')
    for head in header:
        f.write('<th>%s</th>'%head)
    f.write('</tr>\n')
    
    for idd in processedAlerts:
        lst = processedAlerts[idd]
        f.write('<tr class="%s">'%lst['result'])
        f.write('<td>'+lst['id']+'</td>')
        f.write('<td><img src="data:image/png;base64,%s"></td>'%lst['img'])
        f.write('<td>'+lst['type']+'</td>')
        f.write('<td>%s</td>'%lst['result'])
        f.write('<td>%d/%d</td>'%(lst['Npass'], lst['Ntot']))
        f.write('<td>%1.2f</td> <td>%d</td>'%(lst['runS'], lst['date'].timestamp()))
        f.write('</tr>\n')
    
    f.write('''</table>
    <script>
    var el;
    el = document.getElementById("lapprove"); el.innerHTML = "Approve (%d)"; 
    el = document.getElementById("lignore"); el.innerHTML = "Ignore (%d)"; 
    el = document.getElementById("lverify"); el.innerHTML = "Verify (%d)";
    el.click();
    var lst = ['approve','ignore','verify'];
    for(var n=0; n<3; n++) clicked(document.getElementById(lst[n]));
    </script>
    </body></html>
    '''%(counts['Approve'],counts['Ignore'],counts['Verify'])
    )
    
      

HTML file= ./AutoApprovalProc//summaryIndia.html


### Update alerts

In [243]:
import requests

userShow = {
    'mobile_phone': {
        'Approve': (True, json.dumps({"approval_status":"approve", "remarks":"auto approval"})),
        'Ignore': (False, json.dumps({"approval_status":"decline", "remarks":"auto approval"})),
    },
    'seatbelt_violation': {
        'Approve': (True, json.dumps({"approval_status":"approve", "remarks":"auto approval"})),
        'Ignore': (False, json.dumps({"approval_status":"decline", "remarks":"auto approval"})),
    },
}


# ignoreData = {"user_show":False, 
#               "alert_values": json.dumps({"approval_status":"decline", "remarks":"Detection Issue: No yawn/No Eye Close"})
#              }

if True:
    alertUpdateURL = settings['urlUpdateAlerts']
    for aId in processedAlerts:
        ent = processedAlerts[aId]
        if ent['result']=='Verify':
            continue

        accId = settings['accID']
        token = settings['token']
        flag, values = userShow.get(ent['type']).get(ent['result'])

        ignoreData = {'user_show': flag, 'alert_values': values}
        print(aId, typ, ent['result'], flag)# , values)
        r = requests.post(alertUpdateURL.format(alertID=aId,accID=accId,token=token), ignoreData)
        rjd = r.json()


1334833384924381184-mobile_phone-1752018457082 seatbelt_violation Approve True
1334833384924381184-mobile_phone-1752018560582 seatbelt_violation Approve True
1334833384924381184-mobile_phone-1752018933266 seatbelt_violation Approve True
1369971323601682432-mobile_phone-1752021003590 seatbelt_violation Approve True
1359900481819770880-mobile_phone-1752021600119 seatbelt_violation Approve True
1360638833539940352-mobile_phone-1752021722474 seatbelt_violation Approve True
1369971323601682432-mobile_phone-1752021762295 seatbelt_violation Approve True
1369971323601682432-mobile_phone-1752021827725 seatbelt_violation Approve True
1360567581215293440-mobile_phone-1752021882538 seatbelt_violation Approve True
1369971323601682432-mobile_phone-1752022327591 seatbelt_violation Approve True
1369963764438269952-mobile_phone-1752022684012 seatbelt_violation Approve True
1369963764438269952-mobile_phone-1752022764579 seatbelt_violation Approve True
1369963764438269952-mobile_phone-1752022888974 seatb

JSONDecodeError: Expecting value: line 1 column 1 (char 0)